In [8]:
import pandas as pd
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import torch
import math
import os

class AngleDataset(Dataset):
    def __init__(self, csv_file, image_dir):
        self.data = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = os.path.join(self.image_dir, row['filename'])
        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)
        angle_rad = math.radians(row['angle'] % 360)
        target = torch.tensor([math.sin(angle_rad), math.cos(angle_rad)], dtype=torch.float32)
        return image, target

In [9]:
import torch.nn as nn
from torchvision.models import efficientnet_b0, efficientnet_b1, efficientnet_b2, efficientnet_b3

def get_model(version='efficientnet_b0'):
    backbone = {
        'efficientnet_b0': efficientnet_b0,
        'efficientnet_b1': efficientnet_b1,
        'efficientnet_b2': efficientnet_b2,
        'efficientnet_b3': efficientnet_b3,
    }[version](pretrained=True)
    
    in_features = backbone.classifier[1].in_features
    backbone.classifier = nn.Sequential(nn.Linear(in_features, 2))
    return backbone

In [10]:
import torch
import math

def angular_error(pred, target):
    pred_angle = torch.atan2(pred[:, 0], pred[:, 1])
    target_angle = torch.atan2(target[:, 0], target[:, 1])
    diff = torch.abs(pred_angle - target_angle)
    return torch.mean(torch.rad2deg(torch.minimum(diff, 2*math.pi - diff)))

def compute_angle_from_sincos(sin_val, cos_val):
    return (math.degrees(math.atan2(sin_val, cos_val)) + 360) % 360

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch import nn, optim

import os

def train_model(train_csv, val_csv, train_dir, val_dir, version='efficientnet_b0', fine_tune=False, save_path='angle_model.pt'):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_set = AngleDataset(train_csv, train_dir)
    val_set = AngleDataset(val_csv, val_dir)

    train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=32)

    model = get_model(version).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.MSELoss()

    num_epochs = 100 if not fine_tune else 5

    best_maae = float('inf')

    for epoch in range(num_epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss = criterion(pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Evaluation
        model.eval()
        val_loss = 0
        val_angular_error = 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                pred = model(x)
                loss = criterion(pred, y)
                if fine_tune:
                    loss *= 5  # 5x weight during fine-tuning
                val_loss += loss.item()
                val_angular_error += angular_error(pred, y).item()

        avg_maae = val_angular_error / len(val_loader)
        print(f"Epoch {epoch+1}/{num_epochs} | Val Loss: {val_loss:.4f} | Val MAAE: {avg_maae:.2f}")

        # Save best model
        if avg_maae < best_maae:
            best_maae = avg_maae
            torch.save(model.state_dict(), save_path)
            print(f"✅ Saved model to {save_path} (new best MAAE: {avg_maae:.2f})")

train_csv = '/kaggle/input/latlong-dataset/train_combine.csv'
train_dir = '/kaggle/input/latlong-dataset/images_train_combine/'
val_csv = '/kaggle/input/val-dataset/labels_val.csv'
val_dir = '/kaggle/input/val-dataset/images_val'
train_model(train_csv, val_csv, train_dir, val_dir)
# train_model('train.csv', 'val.csv', 'data/train', 'data/val', fine_tune=True)

In [ ]:
import pandas as pd
import torch
from PIL import Image
from torchvision import transforms


def inference(csv_path, img_dir, model_path='angle_model.pt', version='efficientnet_b0'):
    df = pd.read_csv(csv_path)
    model = get_model(version)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])

    output = []
    for _, row in df.iterrows():
        img_path = f"{img_dir}/{row['filename']}"
        img = Image.open(img_path).convert('RGB')
        img = transform(img).unsqueeze(0)
        with torch.no_grad():
            pred = model(img)[0]
        pred_angle = compute_angle_from_sincos(pred[0].item(), pred[1].item())

        entry = {'filename': row['filename'], 'predicted_angle': pred_angle}
        if 'angle' in row:
            actual_angle = row['angle'] % 360
            angle_diff = abs(pred_angle - actual_angle)
            angle_diff = min(angle_diff, 360 - angle_diff)
            entry['maae'] = angle_diff
        output.append(entry)

    pd.DataFrame(output).to_csv('predictions.csv', index=False)

inference(val_csv, val_dir)